In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
train_df = pd.read_csv('train.csv')

In [4]:
test_df = pd.read_csv('test.csv')

In [5]:
features_df = pd.read_csv('features.csv')

In [6]:
stores_df = pd.read_csv('stores.csv')

In [7]:
train_df['Date'] = pd.to_datetime(train_df['Date'])

In [8]:
test_df['Date'] = pd.to_datetime(test_df['Date'])

In [9]:
features_df['Date'] = pd.to_datetime(features_df['Date'])

In [10]:
train_new = pd.merge(train_df, features_df, on=['Store', 'Date', 'IsHoliday'], how = 'left')

In [11]:
train_new = pd.merge(train_new, stores_df, on='Store', how = 'left')

In [12]:
train_new.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,1,1,2010-02-05,24924.50,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
1,1,1,2010-02-12,46039.49,True,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,A,151315
2,1,1,2010-02-19,41595.55,False,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,A,151315
3,1,1,2010-02-26,19403.54,False,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,A,151315
4,1,1,2010-03-05,21827.90,False,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,A,151315


In [13]:
def log_transform_WeeklySales(col):
    Weekly_Sales = col[0]
    
    if Weekly_Sales <= 0:
        return 0
    else:
        return np.log(Weekly_Sales)


In [14]:
#Log transform Weekly sales
train_new['Log_WeeklySales'] = train_new[['Weekly_Sales']].apply(log_transform_WeeklySales,axis=1)

In [15]:
#Function to have a temperature band
def temperature_band(col): 
    Temperature = col[0]
    if (Temperature < 0):
        return 'Less than 0'
    elif (Temperature >= 0) & (Temperature < 20):
        return 'Less than 20'
    elif (Temperature >= 20) & (Temperature < 40):
        return 'Less than 40'
    elif (Temperature >= 40) & (Temperature < 60):
        return 'Less than 60'
    elif (Temperature >= 60) & (Temperature < 70):
        return 'Less than 70'
    elif (Temperature >= 70) & (Temperature < 80):
        return 'Less than 80'
    else:
        return 'Greater than 80'
    

In [16]:
train_new['Temperature Band'] = train_new[['Temperature']].apply(temperature_band,axis=1)

In [17]:
#Function to have a CPI range
def CPI_RANGE(col): 
    CPI = col[0]
    if (CPI >= 125) &(CPI < 130):
        return 'Less than 130'
    elif (CPI >= 130) & (CPI < 135):
        return 'Between 130 and 135'
    elif (CPI >= 135) & (CPI < 140):
        return 'Between 135 and 140'
    elif (CPI >= 140) & (CPI < 145):
        return 'Between 140 and 145'
    elif (CPI >= 145) & (CPI < 190):
        return 'Between 145 and 190'
    elif (CPI >= 190) & (CPI < 210):
        return 'Between 190 and 210'
    elif (CPI >= 210) & (CPI < 230):
        return 'Between 210 and 230'


In [18]:
train_new['CPI_Range'] = train_new[['CPI']].apply(CPI_RANGE,axis=1)

In [19]:
train_new['MarkDown1'].isnull().sum()

270889

In [20]:
train_new['MarkDown2'].isnull().sum()

310322

In [21]:
train_new['MarkDown3'].isnull().sum()

284479

In [22]:
train_new['MarkDown4'].isnull().sum()

286603

In [23]:
train_new['MarkDown5'].isnull().sum()

270138

In [24]:
train_new['Store'].nunique() ## Number of unique stores

45

In [25]:
train_new['Dept'].nunique() ## Count of Depts

81

In [26]:
train_new['Store'].value_counts().head() #record count by stores

13    10474
10    10315
4     10272
1     10244
2     10238
Name: Store, dtype: int64

In [27]:
train_new['Dept'].value_counts().head()

1     6435
10    6435
38    6435
21    6435
67    6435
Name: Dept, dtype: int64

In [28]:
# Create new dataframe by taking only the features from the train_new dataframe that can be used for analysis
weekly_sales = train_new[['Store', 'Dept', 'Date', 'IsHoliday', 'Temperature Band', 'Fuel_Price', 'CPI_Range', 'Unemployment', 'Type', 'Size', 'Weekly_Sales']]

In [29]:
weekly_sales.head()

,Store,Dept,Date,IsHoliday,Temperature Band,Fuel_Price,CPI_Range,Unemployment,Type,Size,Weekly_Sales
0,1,1,2010-02-05,False,Less than 60,2.572,Between 210 and 230,8.106,A,151315,24924.50
1,1,1,2010-02-12,True,Less than 40,2.548,Between 210 and 230,8.106,A,151315,46039.49
2,1,1,2010-02-19,False,Less than 40,2.514,Between 210 and 230,8.106,A,151315,41595.55
3,1,1,2010-02-26,False,Less than 60,2.561,Between 210 and 230,8.106,A,151315,19403.54
4,1,1,2010-03-05,False,Less than 60,2.625,Between 210 and 230,8.106,A,151315,21827.90


In [30]:
weekly_sales.set_index('Date', inplace=True)

In [31]:
weekly_sales.head()

,Store,Dept,IsHoliday,Temperature Band,Fuel_Price,CPI_Range,Unemployment,Type,Size,Weekly_Sales
Date,,,,,,,,,,
2010-02-05,1,1,False,Less than 60,2.572,Between 210 and 230,8.106,A,151315,24924.50
2010-02-12,1,1,True,Less than 40,2.548,Between 210 and 230,8.106,A,151315,46039.49
2010-02-19,1,1,False,Less than 40,2.514,Between 210 and 230,8.106,A,151315,41595.55
2010-02-26,1,1,False,Less than 60,2.561,Between 210 and 230,8.106,A,151315,19403.54
2010-03-05,1,1,False,Less than 60,2.625,Between 210 and 230,8.106,A,151315,21827.90


### Min, max, mean, median, std of sales 3 Weeks, 6 Weeks, 9 Weeks before your date per department / store

In [32]:
## Previous 3 Weeks, 6 Weeks, 9 Weeks (lag)
weekly_sales['t-3'] = weekly_sales.loc[:,'Weekly_Sales'].shift(3)


C:\Users\b003373\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [33]:
weekly_sales.head()

,Store,Dept,IsHoliday,Temperature Band,Fuel_Price,CPI_Range,Unemployment,Type,Size,Weekly_Sales,t-3
Date,,,,,,,,,,,
2010-02-05,1,1,False,Less than 60,2.572,Between 210 and 230,8.106,A,151315,24924.50,NaN
2010-02-12,1,1,True,Less than 40,2.548,Between 210 and 230,8.106,A,151315,46039.49,NaN
2010-02-19,1,1,False,Less than 40,2.514,Between 210 and 230,8.106,A,151315,41595.55,NaN
2010-02-26,1,1,False,Less than 60,2.561,Between 210 and 230,8.106,A,151315,19403.54,24924.50
2010-03-05,1,1,False,Less than 60,2.625,Between 210 and 230,8.106,A,151315,21827.90,46039.49


In [34]:
weekly_sales['t-6'] = weekly_sales.loc[:,'Weekly_Sales'].shift(6)
weekly_sales['t-9'] = weekly_sales.loc[:,'Weekly_Sales'].shift(9)

C:\Users\b003373\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\b003373\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [35]:
weekly_sales.head(10)

,Store,Dept,IsHoliday,Temperature Band,Fuel_Price,CPI_Range,Unemployment,Type,Size,Weekly_Sales,t-3,t-6,t-9
Date,,,,,,,,,,,,,
2010-02-05,1,1,False,Less than 60,2.572,Between 210 and 230,8.106,A,151315,24924.50,NaN,NaN,NaN
2010-02-12,1,1,True,Less than 40,2.548,Between 210 and 230,8.106,A,151315,46039.49,NaN,NaN,NaN
2010-02-19,1,1,False,Less than 40,2.514,Between 210 and 230,8.106,A,151315,41595.55,NaN,NaN,NaN
2010-02-26,1,1,False,Less than 60,2.561,Between 210 and 230,8.106,A,151315,19403.54,24924.50,NaN,NaN
2010-03-05,1,1,False,Less than 60,2.625,Between 210 and 230,8.106,A,151315,21827.90,46039.49,NaN,NaN
2010-03-12,1,1,False,Less than 60,2.667,Between 210 and 230,8.106,A,151315,21043.39,41595.55,NaN,NaN
2010-03-19,1,1,False,Less than 60,2.720,Between 210 and 230,8.106,A,151315,22136.64,19403.54,24924.50,NaN
2010-03-26,1,1,False,Less than 60,2.732,Between 210 and 230,8.106,A,151315,26229.21,21827.90,46039.49,NaN
2010-04-02,1,1,False,Less than 70,2.719,Between 210 and 230,7.808,A,151315,57258.43,21043.39,41595.55,NaN


In [36]:
## Min for previous 3 Weeks

weekly_sales['Min(t-1,t-2,t-3)'] = weekly_sales['Weekly_Sales'].shift(1).rolling(window=3).min()

weekly_sales.head(10)


C:\Users\b003373\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Store,Dept,IsHoliday,Temperature Band,Fuel_Price,CPI_Range,Unemployment,Type,Size,Weekly_Sales,t-3,t-6,t-9,"Min(t-1,t-2,t-3)"
Date,,,,,,,,,,,,,,
2010-02-05,1,1,False,Less than 60,2.572,Between 210 and 230,8.106,A,151315,24924.50,NaN,NaN,NaN,NaN
2010-02-12,1,1,True,Less than 40,2.548,Between 210 and 230,8.106,A,151315,46039.49,NaN,NaN,NaN,NaN
2010-02-19,1,1,False,Less than 40,2.514,Between 210 and 230,8.106,A,151315,41595.55,NaN,NaN,NaN,NaN
2010-02-26,1,1,False,Less than 60,2.561,Between 210 and 230,8.106,A,151315,19403.54,24924.50,NaN,NaN,24924.50
2010-03-05,1,1,False,Less than 60,2.625,Between 210 and 230,8.106,A,151315,21827.90,46039.49,NaN,NaN,19403.54
2010-03-12,1,1,False,Less than 60,2.667,Between 210 and 230,8.106,A,151315,21043.39,41595.55,NaN,NaN,19403.54
2010-03-19,1,1,False,Less than 60,2.720,Between 210 and 230,8.106,A,151315,22136.64,19403.54,24924.50,NaN,19403.54
2010-03-26,1,1,False,Less than 60,2.732,Between 210 and 230,8.106,A,151315,26229.21,21827.90,46039.49,NaN,21043.39
2010-04-02,1,1,False,Less than 70,2.719,Between 210 and 230,7.808,A,151315,57258.43,21043.39,41595.55,NaN,21043.39


In [37]:
weekly_sales['Min(t-1 to t-6)'] = weekly_sales['Weekly_Sales'].shift(1).rolling(window=6).min()
weekly_sales['Min(t-1 to t-9)'] = weekly_sales['Weekly_Sales'].shift(1).rolling(window=9).min()
weekly_sales.head(10)

C:\Users\b003373\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\b003373\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Store,Dept,IsHoliday,Temperature Band,Fuel_Price,CPI_Range,Unemployment,Type,Size,Weekly_Sales,t-3,t-6,t-9,"Min(t-1,t-2,t-3)",Min(t-1 to t-6),Min(t-1 to t-9)
Date,,,,,,,,,,,,,,,,
2010-02-05,1,1,False,Less than 60,2.572,Between 210 and 230,8.106,A,151315,24924.50,NaN,NaN,NaN,NaN,NaN,NaN
2010-02-12,1,1,True,Less than 40,2.548,Between 210 and 230,8.106,A,151315,46039.49,NaN,NaN,NaN,NaN,NaN,NaN
2010-02-19,1,1,False,Less than 40,2.514,Between 210 and 230,8.106,A,151315,41595.55,NaN,NaN,NaN,NaN,NaN,NaN
2010-02-26,1,1,False,Less than 60,2.561,Between 210 and 230,8.106,A,151315,19403.54,24924.50,NaN,NaN,24924.50,NaN,NaN
2010-03-05,1,1,False,Less than 60,2.625,Between 210 and 230,8.106,A,151315,21827.90,46039.49,NaN,NaN,19403.54,NaN,NaN
2010-03-12,1,1,False,Less than 60,2.667,Between 210 and 230,8.106,A,151315,21043.39,41595.55,NaN,NaN,19403.54,NaN,NaN
2010-03-19,1,1,False,Less than 60,2.720,Between 210 and 230,8.106,A,151315,22136.64,19403.54,24924.50,NaN,19403.54,19403.54,NaN
2010-03-26,1,1,False,Less than 60,2.732,Between 210 and 230,8.106,A,151315,26229.21,21827.90,46039.49,NaN,21043.39,19403.54,NaN
2010-04-02,1,1,False,Less than 70,2.719,Between 210 and 230,7.808,A,151315,57258.43,21043.39,41595.55,NaN,21043.39,19403.54,NaN


In [38]:
## Mean, median, min and max for previous 3 weeks, 6 weeks, 9 weeks

weekly_sales['Max(t-1,t-2,t-3)'] = weekly_sales['Weekly_Sales'].shift(1).rolling(window=3).max()
weekly_sales['Mean(t-1,t-2,t-3)'] = weekly_sales['Weekly_Sales'].shift(1).rolling(window=3).mean()
weekly_sales['Median(t-1,t-2,t-3)'] = weekly_sales['Weekly_Sales'].shift(1).rolling(window=3).median()

weekly_sales['Max(t-1 to t-6)'] = weekly_sales['Weekly_Sales'].shift(1).rolling(window=6).max()
weekly_sales['Mean(t-1 to t-6)'] = weekly_sales['Weekly_Sales'].shift(1).rolling(window=6).mean()
weekly_sales['Median(t-1 to t-6)'] = weekly_sales['Weekly_Sales'].shift(1).rolling(window=6).median()

weekly_sales['Max(t-1 to t-9)'] = weekly_sales['Weekly_Sales'].shift(1).rolling(window=9).max()
weekly_sales['Mean(t-1 to t-9)'] = weekly_sales['Weekly_Sales'].shift(1).rolling(window=9).mean()
weekly_sales['Median(t-1 to t-9)'] = weekly_sales['Weekly_Sales'].shift(1).rolling(window=9).median()



C:\Users\b003373\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\b003373\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [41]:
weekly_sales.head()

,Store,Dept,IsHoliday,Temperature Band,Fuel_Price,CPI_Range,Unemployment,Type,Size,Weekly_Sales,...,Min(t-1 to t-9),"Max(t-1,t-2,t-3)","Mean(t-1,t-2,t-3)","Median(t-1,t-2,t-3)",Max(t-1 to t-6),Mean(t-1 to t-6),Median(t-1 to t-6),Max(t-1 to t-9),Mean(t-1 to t-9),Median(t-1 to t-9)
Date,,,,,,,,,,,,,,,,,,,,,
2010-02-05,1,1,False,Less than 60,2.572,Between 210 and 230,8.106,A,151315,24924.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-02-12,1,1,True,Less than 40,2.548,Between 210 and 230,8.106,A,151315,46039.49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-02-19,1,1,False,Less than 40,2.514,Between 210 and 230,8.106,A,151315,41595.55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-02-26,1,1,False,Less than 60,2.561,Between 210 and 230,8.106,A,151315,19403.54,...,NaN,46039.49,37519.846667,41595.55,NaN,NaN,NaN,NaN,NaN,NaN
2010-03-05,1,1,False,Less than 60,2.625,Between 210 and 230,8.106,A,151315,21827.90,...,NaN,46039.49,35679.526667,41595.55,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
weekly_sales.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 421570 entries, 2010-02-05 to 2012-10-26
Data columns (total 25 columns):
Store                  421570 non-null int64
Dept                   421570 non-null int64
IsHoliday              421570 non-null bool
Temperature Band       421570 non-null object
Fuel_Price             421570 non-null float64
CPI_Range              421570 non-null object
Unemployment           421570 non-null float64
Type                   421570 non-null object
Size                   421570 non-null int64
Weekly_Sales           421570 non-null float64
t-3                    421567 non-null float64
t-6                    421564 non-null float64
t-9                    421561 non-null float64
Min(t-1,t-2,t-3)       421567 non-null float64
Min(t-1 to t-6)        421564 non-null float64
Min(t-1 to t-9)        421561 non-null float64
Max(t-1,t-2,t-3)       421567 non-null float64
Mean(t-1,t-2,t-3)      421567 non-null float64
Median(t-1,t-2,t-3)    421567 non-null 

In [42]:
weekly_sales[['Store', 'Weekly_Sales', 't-3', 'Min(t-1,t-2,t-3)', 'Max(t-1,t-2,t-3)', 'Mean(t-1,t-2,t-3)','Median(t-1,t-2,t-3)']]

,Store,Weekly_Sales,t-3,"Min(t-1,t-2,t-3)","Max(t-1,t-2,t-3)","Mean(t-1,t-2,t-3)","Median(t-1,t-2,t-3)"
Date,,,,,,,
2010-02-05,1,24924.50,NaN,NaN,NaN,NaN,NaN
2010-02-12,1,46039.49,NaN,NaN,NaN,NaN,NaN
2010-02-19,1,41595.55,NaN,NaN,NaN,NaN,NaN
2010-02-26,1,19403.54,24924.50,24924.50,46039.49,37519.846667,41595.55
2010-03-05,1,21827.90,46039.49,19403.54,46039.49,35679.526667,41595.55
2010-03-12,1,21043.39,41595.55,19403.54,41595.55,27608.996667,21827.90
2010-03-19,1,22136.64,19403.54,19403.54,21827.90,20758.276667,21043.39
2010-03-26,1,26229.21,21827.90,21043.39,22136.64,21669.310000,21827.90
2010-04-02,1,57258.43,21043.39,21043.39,26229.21,23136.413333,22136.64
